Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# New Section

# New Section

  Created wheel for mmh3: filename=mmh3-2.5.1-cp36-cp36m-linux_x86_64.whl size=37856 sha256=e2048396341710395c189c3b01ca08d97707e737d47e0a02b4bb3c26019bebfe
  Stored in directory: /root/.cache/pip/wheels/38/b4/ea/6e4e321c625d3320c0c496bf4088371546d8fce5f1dd71b219
Successfully built mmh3


In [1]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [19]:
class Differential_Privacy_CMS():
    def __init__(self, width, depth,epsilon):
        ''' Method to initialize the data structure
        @param width int: Width of the table
        @param depth int: Depth of the table (num of hash func)
        @param epsilon: privacy parameter
        In this case, when we declare differential privacy class instance, we give the parameters m and k, or width and depth
        '''
        self.width = width #hash values range between 0 to width-1
        self.depth = depth #number of hash functions
        self.epsilon = epsilon 
        self.seed = np.random.choice(np.arange(0,self.width),replace=False,size=self.depth)
    #Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
#we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
#and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
#result vector. 

    def CLient_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(self.seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 

    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        #print(M)
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        #print('hi',M)
        return M 

    def Server_Side(self,Sketch_Matrix,d,length):
        '''It returns the estimated frequency of a data element given to it. So it has two parameters, data element and the length of the 
        data stream we are considering'''
        n = length 
        frac1 = self.width/(self.width-1)
        frac2 = n/self.width
        row_sum = 0
        for i in range(self.depth):
            hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
            int_hex = int(hex_num[:5],16)
            index = int_hex % self.width
            row_sum = row_sum + Sketch_Matrix[i,index]
        avg_row_sum = row_sum/self.depth 
        subtraction = avg_row_sum - frac2 
        assumed_freq = frac1*subtraction 
        return assumed_freq
        
    def Count_Mean_Sketch(self,D_s,D): 
        '''D_s is the stream of data and this is a subset of universe of data'''
        Modified_datalist = []
        Sketch_Matrix = []
        freq_vect = {}
        length = len(D_s) 
        for elt in enumerate(D_s):
            Modified_datalist.append(self.CLient_Side(elt[1])) 
        Sketch_Matrix = self.Compute_Sketch_Matrix(Modified_datalist)
        for d in D:
            freq_vect[d] = self.Server_Side(Sketch_Matrix,d,length) 
        #for d in freq_vect:
          
            #freq_vect[d]=round(freq_vect[d])
        return freq_vect

In [20]:
path_list = [(0,1),(0,7),(0,7),(0,3),(0,8),(0,10),(1,1),(1,1),(1,1),(8,10),(8,10),(8,20),(7,10)]

In [21]:
set(path_list)

{(0, 1), (0, 3), (0, 7), (0, 8), (0, 10), (1, 1), (7, 10), (8, 10), (8, 20)}

In [22]:
path_list.count((0,6))

0

In [58]:
def frequency_counter_on_average(path_list,epsilon,run_number):
    '''This function runs the entire process multiple and take the running average of frequencies for reducing error'''
    u_list = [(x,y) for x in range(30) for y in range(30)]
    freq_counter_cum = {d:0 for d in u_list}
    for i in range(run_number):
        class_instance = Differential_Privacy_CMS(100,100,epsilon)
        freq_counter_new = class_instance.Count_Mean_Sketch(path_list, u_list)
        for d in freq_counter_new:
            freq_counter_cum[d]=(i*freq_counter_cum[d]+freq_counter_new[d])/(i+1)
    an_list =[]
    for d in freq_counter_cum: 
        if freq_counter_cum[d]>=1 and d in path_list:
            an_list.append((d, freq_counter_cum[d], path_list.count(d)))
    return freq_counter_cum, an_list
big_dict1 = frequency_counter_on_average(path_list,5,500)


In [59]:
elt_list = []
big_dict1[0]
for elt in big_dict1[0]:
    if big_dict1[0][elt]>=1:
        elt_list.append(elt)

In [60]:
set(path_list) & set(elt_list)

{(0, 1), (0, 7), (0, 8), (0, 10), (1, 1), (8, 10)}

In [61]:
#This method will use the same set of hash functions, and the class instance will be created only one time
def frequency_counter_on_average1(path_list, epsilon, run_numer):
    u_list = [(x,y) for x in range(30) for y in range(30)]
    freq_counter_num = {d:0 for d in u_list}
    class_instance = Differential_Privacy_CMS(100, 100, epsilon)
    skecth_matrix = np.zeros((100, 100))
    for i in range(run_numer):
        D = []
        for elt in path_list:
            D.append(class_instance.CLient_Side(elt))
        M = class_instance.Compute_Sketch_Matrix(D)
        for row in range(100):
            for col in range(100):
                skecth_matrix[row, col]= (i*skecth_matrix[row, col]+M[row, col])/(i+1)
    l = len(path_list)
    big_freq_dict = {}
    for elt in u_list:
        big_freq_dict[elt]=class_instance.Server_Side(skecth_matrix, elt, l)
    return big_freq_dict

big_dict = frequency_counter_on_average1(path_list, 5, 500)

        


In [62]:
elt_list2 = []
for elt in big_dict:
    if big_dict[elt]>=1:
        elt_list2.append(elt)

In [63]:
set(path_list) & set(elt_list2)

{(0, 1), (0, 3), (0, 7), (0, 10), (1, 1), (8, 10)}

In [ ]:
path_list = [(15, 9), (16, 9), (17, 9), (18, 9), (19, 9), (19, 10), (19, 9), (18, 9), (18, 8), (18, 7), (18, 8), (18, 9), (19, 9), (19, 8), (19, 9), (19, 8), (18, 8), (18, 7), (18, 6), (18, 5), (17, 5), (18, 5), (17, 5), (16, 5), (17, 5), (16, 5), (16, 4), (16, 3), (15, 3), (15, 2), (14, 2), (14, 1), (13, 1), (12, 1), (13, 1), (13, 0), (12, 0), (11, 0), (12, 0), (11, 0), (10, 0), (9, 0), (8, 0), (7, 0), (7, 1), (7, 0), (6, 0), (6, 1), (5, 1), (5, 0), (5, 1), (6, 1), (6, 0), (5, 0), (4, 0), (3, 0), (2, 0), (2, 1), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 2), (6, 1), (7, 1), (8, 1), (7, 1), (8, 1), (8, 2), (8, 1), (7, 1), (7, 2), (6, 2), (6, 1), (5, 1), (4, 1), (3, 1), (4, 1), (3, 1), (4, 1), (4, 2), (3, 2), (3, 3), (2, 3), (2, 2), (1, 2), (1, 1), (0, 1), (1, 1), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 5), (3, 6), (3, 7), (3, 6), (2, 6), (2, 5), (1, 5), (0, 5), (1, 5), (1, 4), (1, 3), (2, 3), (2, 2), (1, 2), (2, 2), (2, 1), (2, 2), (1, 2), (2, 2), (2, 3), (1, 3), (1, 4), (1, 5), (0, 5), (0, 6), (0, 5), (0, 6), (0, 5), (1, 5), (1, 6), (0, 6), (1, 6), (0, 6), (1, 6), (1, 7), (2, 7), (1, 7), (0, 7), (1, 7), (2, 7), (1, 7), (1, 6), (0, 6), (0, 5), (0, 6), (0, 7), (0, 6), (1, 6), (1, 5), (1, 6), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (0, 10), (1, 10), (1, 9), (0, 9), (1, 9), (1, 10), (2, 10), (1, 10), (1, 9), (1, 8), (1, 7), (1, 6), (1, 5), (2, 5), (1, 5), (0, 5), (0, 4), (0, 3), (1, 3), (0, 3), (0, 4), (1, 4), (1, 3), (0, 3), (1, 3), (1, 4), (1, 3), (1, 2), (2, 2), (3, 2), (4, 2), (4, 1), (5, 1), (5, 2)]

In [ ]:
frequency_counter_on_average(path_list,5,400)

In [ ]:
def root_mean_squared_error(predicted, expected):
    '''we are getting one dictionary of predicted frequencies and the genuine list'''
    sum = 0
    l = len(predicted)
    unique_elt = set(expected)
    for elt in unique_elt:
        sum+=(predicted.get(elt,0)-expected.count(elt))**2
    mse = sum/l
    return math.sqrt(mse)
       

In [ ]:
def epsilon_evaluator(path_list,epsilon_list):
    '''This function calls frequency average multiple times with different epsilons to see which value gives the value of epsilon gives the minimum error and then plot it'''
    eps_err_dict = {}
    for epsilon in epsilon_list:
        predicted = frequency_counter_on_average(path_list,epsilon,300)
        error = root_mean_squared_error(predicted, path_list)
        eps_err_dict[epsilon] = error
    return eps_err_dict
x = np.array([1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9])
y = np.exp(x)
y_list = y.tolist()
print(epsilon_evaluator(path_list,y_list))

{2.718281828459045: 1.0044777621531198, 3.0041660239464334: 0.886639757155907, 3.3201169227365472: 0.877882406053931, 3.6692966676192444: 0.8491281903323685, 4.0551999668446745: 0.7318999165861447, 4.4816890703380645: 0.5961974731351187, 4.953032424395115: 0.5777940086716824, 5.4739473917272: 0.5286519766295436, 6.0496474644129465: 0.5198424981165574, 6.6858944422792685: 0.475201528845881}


In [ ]:
path_list = [ 96, 121,  38, 109,  81,  83,  10,  71, 191, 162,  79, 198, 136,
       147,  95,  68, 152, 151,  72, 165, 154,  50,  66, 131, 123, 161,
        70,  12, 117,  41, 148, 150,  69,  88,   6,  64, 196,  23, 172,
       144, 142,  75, 184,  59,  34,  52,  45,  54,  19, 156, 137, 158,
        18, 113,  80, 135,  32,  30,  20,  29,  49,  22, 177,  91,  78,
        60,  37,  90, 133, 103, 146, 169,  63, 114,  82, 194, 138, 168,
       197,   3,  51,  86, 140,  65,  55, 187, 160, 149,   0,  33, 130,
       193, 122,  77, 108,  17, 183,  31, 173,  36]

In [ ]:
def run_number_evaluator(path_list,epsilon):
    run_number_dict = {}
    for run_number in range(50,650,50):
        predicted = frequency_counter_on_average(path_list,epsilon,run_number)
        error = root_mean_squared_error(predicted,path_list)
        run_number_dict[run_number] = error
        print(run_number,error)
    return run_number_dict 
run_number_evaluator(path_list,5)